In [38]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# secret info
client_id = 'X'
client_secret = 'X'
redirect_uri='http://localhost:8000'
scope = 'user-library-read user-top-read user-read-recently-played'

# authenticate with Spotify API
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope=scope))


# retrieve a user's top tracks from Spotify
def get_user_top_tracks(username, time_range='medium_term', limit=20):
    results = sp.current_user_top_tracks(time_range=time_range, limit=limit)
    top_tracks = results['items']
    while results['next']:
        results = sp.next(results)
        top_tracks.extend(results['items'])
    track_ids = [track['id'] for track in top_tracks]
    return track_ids

# get audio features
def get_audio_features(track_ids):
    audio_features = []
    for i in range(0, len(track_ids), 100):
        batch_ids = track_ids[i:i+100]
        batch_features = sp.audio_features(batch_ids)
        audio_features += batch_features
    relevant_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
    features = []
    for track in audio_features:
        track_features = [track[feature] for feature in relevant_features]
        features.append(track_features)
    return features

def normalize(features):
    norm = np.linalg.norm(features)
    if norm == 0:
        return features
    return features / norm

def calculate_similarity(features_1, features_2):
    features_1_normalized = normalize(features_1)
    features_2_normalized = normalize(features_2)
    dot_product = np.dot(features_1_normalized, features_2_normalized.T)
    return dot_product[0]

def get_user_tracks(username):
    track_ids = []
    results = sp.current_user_saved_tracks()
    while results and results['items']:
        for item in results['items']:
            track_ids.append(item['track']['id'])
        results = sp.next(results)
    return track_ids

def generate_song_recommendations(username):
    # Get the user's tracks
    user_track_ids = get_user_tracks(username)
    # Get audio features for the user's tracks
    user_features = get_audio_features(user_track_ids)
    # Get audio features for all tracks in Spotify's catalog
    catalog_tracks = sp.search(q='year:2022', type='track', limit=50)['tracks']['items']
    catalog_track_ids = [track['id'] for track in catalog_tracks]
    catalog_track_features = get_audio_features(catalog_track_ids)
    # Calculate cosine similarity between the user's tracks and all tracks in Spotify's catalog
    similarities = []
    for track_features in user_features:
        track_similarity = calculate_similarity(track_features, catalog_track_features)
        similarities.append(track_similarity)
    # Sort the tracks by similarity and return the top 10 recommendations
    sorted_tracks = sorted(zip(catalog_tracks, similarities), key=lambda x: x[1], reverse=True)
    top_tracks = sorted_tracks[:10]
    top_track_names = [track['name'] for track, _ in top_tracks]
    return top_track_names


# call the generate_recommendations function for a Spotify user
username = 'your_spotify_username'
recommendations = generate_song_recommendations(username)
print(recommendations)

# define a function to generate artist recommendations for a user based on content-based filtering
def generate_artist_recommendations(username):
    # Get the user's top tracks
    user_track_ids = get_user_top_tracks(username)
    # Get audio features for the user's top tracks
    user_features = get_audio_features(user_track_ids)
    # Get audio features for all artists in Spotify's catalog
    catalog_artists = sp.search(q='year:2022', type='artist', limit=50)['artists']['items']
    catalog_artist_ids = [artist['id'] for artist in catalog_artists]
    catalog_artist_features = []
    for i in range(0, len(catalog_artist_ids), 50):
        batch_ids = catalog_artist_ids[i:i+50]
        batch_artists = sp.artists(batch_ids)['artists']
        for artist in batch_artists:
            artist_tracks = sp.artist_top_tracks(artist['id'], country='US')['tracks'][:5]
            artist_track_ids = [track['id'] for track in artist_tracks]
            artist_features = get_audio_features(artist_track_ids)
            catalog_artist_features.append({
                'id': artist['id'],
                'name': artist['name'],
                'features': artist_features
            })
    # calculate cosine similarity between the user's top tracks and all artists in Spotify's catalog
    similarities = []
    for catalog_artist in catalog_artist_features:
        artist_features = catalog_artist['features']
        artist_similarity = 0
        for track_features in user_features:
            for artist_track_features in artist_features:
                track_similarity = calculate_similarity([track_features], [artist_track_features])
                artist_similarity += track_similarity
        artist_similarity /= len(user_features) * len(artist_features)
        similarities.append(artist_similarity)
    # sort the artists by similarity and return the top 10 recommendations
    sorted_artists = sorted(zip(catalog_artist_features, similarities), key=lambda x: x[1], reverse=True)
    top_artists = sorted_artists[:10]
    top_artist_names = [artist['name'] for artist, _ in top_artists]
    return top_artist_names

# call the generate_artist_recommendations function to get a list of recommended artist names
recommended_artists = generate_artist_recommendations(username)

# list the recommended artist names
print("Recommended Artists:")
for artist_name in recommended_artists:
    print(artist_name)

["Creepin' (with The Weeknd & 21 Savage)", 'As It Was', 'Out thë way', 'Niagara Falls (Foot or 2) [with Travis Scott & 21 Savage]', 'Superhero (Heroes & Villains) [with Future & Chris Brown]', 'Anti-Hero', "I'm Good (Blue)", 'Nonsense', 'ALL MINE', 'Thought You Should Know']
Recommended Artists:
Yeat
Lil Baby
Gunna
SZA
Ariana Grande
Steve Lacy
Nicki Minaj
Polo G
A Boogie Wit da Hoodie
Doja Cat


ModuleNotFoundError: No module named 'your_recommendation_module'